In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd

from importlib import reload
from joebvp import cfg, utils, VPmeasure

read_sets: Using set file -- 
  C:\Users\Billy\AppData\Local\Programs\Python\Python312\Lib\site-packages\linetools\lists\sets\llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   C:\Users\Billy\AppData\Local\Programs\Python\Python312\Lib\site-packages\linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   C:\Users\Billy\AppData\Local\Programs\Python\Python312\Lib\site-packages\linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   C:\Users\Billy\AppData\Local\Programs\Python\Python312\Lib\site-packages\linetools/data/lines/morto

In [4]:
def replace_string(input_str):
    segment = """lt_xspec [dataset_name]_x1d.fits 
lt_continuumfit --redshift 0.0009 [dataset_name]_x1d.fits [dataset_name]_continuumfit.fits
pyigm_igmguesses [dataset_name]_continuumfit.fits -o [dataset_name]_x1dfits_model.json
pyigm_fitdla --out_file [dataset_name]_xfitdla.json [dataset_name]_continuumfit.fits 0"""
    
    modified_segment = segment.replace("[dataset_name]", input_str)
    print(modified_segment)
    
replace_string("oda973030")

lt_xspec oda973030_x1d.fits 
lt_continuumfit --redshift 0.0009 oda973030_x1d.fits oda973030_continuumfit.fits
pyigm_igmguesses oda973030_continuumfit.fits -o oda973030_x1dfits_model.json
pyigm_fitdla --out_file oda973030_xfitdla.json oda973030_continuumfit.fits 0


In [5]:
reload(cfg)

d = pd.DataFrame({
    'instr': ['COS', 'COS', 'COS', 'COS', 'STIS', 'STIS', 'Gaussian'],
    'gratings': ['G130M', 'G160M', 'G185M', 'G225M', 'E230M', 'E140M', 'N/A'],
    'slits': ['NA', 'NA', 'NA', 'NA', '0.2x0.2', '0.2x0.2', 'NA'],
    'lsfranges': [[1100, 1460], [1400, 1800], [1800, 2100], [2100, 2278], [1607, 3129], [1144, 1729], [899, 1191]],
    'lps': ['1', '1', '1', '1', '1', '1', 'NA'],
    'cen_wave': ['1291', '1611', '1953', '2250', '1978', '1425', 'NA'],
    'pixel_scales': [None, None, None, None, None, None, 0.013], 
    'fwhms': [None, None, None, None, None, None, 0.02]
})

num = 5

cfg.lsfs = []
for col in ['instr', 'gratings', 'slits', 'cen_wave', 'pixel_scales', 'fwhms']:
    setattr(cfg, col, [d.at[num, col]])

cfg.lsfranges = np.array([d.at[num, 'lsfranges']])

In [6]:
base_directory = "C:/Users/Billy/Desktop/float of dawn/PGMW3223/E140M"
directories = sorted([d for d in os.listdir(base_directory) if d.startswith("E140M")])

In [7]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    igm_models = glob.glob(os.path.join(current_directory, "*_x1dfits_model.json"))
    continuumfits = glob.glob(os.path.join(current_directory, "*_continuumfit.fits"))

    utils.pyigm_to_veeper(igm_models[0], continuumfits[0])

    os.chdir(base_directory)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [11]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    component_groups_dir = os.path.join(current_directory, "component_groups")
    output_dir = os.path.join(current_directory, "modified_component_groups")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    mc_lines_file = "C:/Users/Billy/Desktop/float of dawn/mc_lines_metal_e140m.txt"
    mc_lines_df = pd.read_csv(mc_lines_file, sep = r"\s+", header = None, names = ["line", "wavelength"])
    mc_lines_df["wavelength"] = mc_lines_df["wavelength"].astype(float)

    def is_valid_line(row, line_df):
        trans = row["trans"].strip()
        restwave = row["restwave"]
        matching_lines = line_df[line_df["line"] == trans]
        for wave in matching_lines["wavelength"]:
            if 0.997 * wave <= restwave <= 1.003 * wave:
                return True
        return False

    input_files = glob.glob(os.path.join(component_groups_dir, "*.txt"))

    for file in input_files:
        df = pd.read_csv(file, sep = "|")
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
        df["restwave"] = pd.to_numeric(df["restwave"], errors = "coerce")
        df["bval"] = pd.to_numeric(df["bval"], errors = "coerce")
    
        valid_mask = df.apply(lambda row: is_valid_line(row, mc_lines_df), axis = 1)
        df_filtered = df[valid_mask].copy()
    
        # df_filtered["col"] = df_filtered["col"] - 0.2
        df_filtered["bval"] = df_filtered["bval"] / 2
    
        output_file = os.path.join(output_dir, os.path.basename(file))
        df_filtered.to_csv(output_file, sep = "|", index = False)
        # print(f"Processed file: {os.path.basename(file)} -> {output_file}")

In [12]:
# Before fitting, should manually adjust the input group files

In [13]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    igm_models = glob.glob(os.path.join(current_directory, "*_x1dfits_model.json"))
    continuumfits = glob.glob(os.path.join(current_directory, "*_continuumfit.fits"))

    component_groups_dir = os.path.join(current_directory, "modified_component_groups")
    
    input_group_files = sorted([f for f in glob.glob(os.path.join(component_groups_dir, "input_group_*.txt")) if re.match(r".*input_group_\d+\.txt$", f)])

    VPmeasure.batch_fit(continuumfits[0], input_group_files, filepath = './modified_component_groups/')

    os.chdir(base_directory)

Bad METADATA;  proceeding without
`ftol` termination condition is satisfied.
Function evaluations 12, initial cost 2.0032e+02, final cost 4.5562e+01, first-order optimality 1.88e-03.

Fit results: 

1608.45	 0.000910	 15.405	 18.900	 0.39047
 	  	  	 0.104	 1.07	 1.012 

1611.2	 0.000910	 15.405	 18.900	 0.39047
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.990468
Iteration 1 -
`ftol` termination condition is satisfied.
Function evaluations 12, initial cost 2.0032e+02, final cost 4.5562e+01, first-order optimality 1.88e-03.

Fit results: 

1608.45	 0.000910	 15.405	 18.900	 0.39047
 	  	  	 0.104	 1.07	 1.012 

1611.2	 0.000910	 15.405	 18.900	 0.39047
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.990468
Iteration 2 -
Fit converged after 2 iterations.
VPmeasure: Fit converged: C:/Users/Billy/Desktop/float of dawn/PGMW3223/E140M\E140M10\modified_component_groups\input_group_0.txt
Line parameters written to:
C:/Users/Billy/Desktop/float of dawn/PGMW3223/E140M\E140M10\modified_compo

In [14]:
for spectra_directory in directories:

    current_directory = os.path.join(base_directory, spectra_directory)

    os.chdir(current_directory)

    df = pd.read_csv('compiledVPoutputs.dat', sep = '|')
    filtered_df = df[df['zsys'] > 0.0004]
    filtered_df = filtered_df[filtered_df['sigcol'] != 0.000]
    result = filtered_df[['col', 'sigcol', 'trans']]

    unique_result = result.drop_duplicates()
    print(unique_result)
    os.chdir(base_directory)

      col  sigcol trans
0  15.405   0.104  FeII
2  16.170   0.047  MgII
4  15.752   0.071  S II
6  13.824   0.036  NiII
      col  sigcol trans
0  16.581   0.274  MgII
2  16.227   0.340  S II
4  13.921   0.033  NiII
7  12.866   0.100  CuII
      col  sigcol trans
0  15.404   0.109  FeII
2  16.366   0.090  MgII
4  16.021   0.192  S II
6  13.937   0.035  NiII


In [ ]:
# END